# Chapter 4: Multiple StepTypes

To avoid the soundness error from `evil_witness` in Chapter 3, we need to constrain that forward signals "a" and "b" are both 1 in the first step instance. Because this isn't true for all step instances, we need to create a new step type, called `FiboFirstStep`, which is idential to `FiboStep` except two additional constraints for "a" and "b":

| Step Type | Step Instance Index || Signals |||| Setups |||
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
||| a | b | c | constraint 1 | constraint 2 | constraint 3 | constraint 4 | constraint 5 |
| fibo first step | 0 | 1 | 1 | 2 | a + b == c | b == a.next | c == b.next | a == 1 | b == 1 |
| fibo step | 1 | 1 | 2 | 3 | a + b == c | b == a.next | c == b.next | n.a. | n.a. |
| fibo step | 2 | 2 | 3 | 5 | a + b == c | b == a.next | c == b.next | n.a. | n.a. |
| fibo step | 3 | 3 | 5 | 8 | a + b == c | b == a.next | c == b.next | n.a. | n.a. |
| ... | ... || ... |||| ... |||

The following shows the incremental code of `FiboFirstStep` compared to `FiboStep`:

```python
class FiboFirstStep(StepType):
    def setup(self):
        # ...
        self.constr(eq(self.circuit.a, 1))
        self.constr(eq(self.circuit.b, 1))

    def wg(self, args):
        # ...
```

Next, in circuit `setup`, we need to append this step type and constrain that the first step instance is `FiboFirstStep`. Otherwise `setup` is identical to that in Chapter 3:

```python
class Fibonacci(Circuit):
    def setup(self):
        # ...
        self.fibo_first_step = self.step_type(FiboFirstStep(self, "fibo_first_step"))
        self.pragma_first_step(self.fibo_first_step)
        
    def trace(self, args):
        # TODO
```

Finally, in circuit `trace`, we need to instantiate `FiboFirstStep` for the first step instance. Otherwise `trace` is identical to that in Chapter 3:

```python
class Fibonacci(Circuit):
    def setup(self):
        # ...
        
    def trace(self, args):
        self.add(self.fibo_first_step, (1, 1))
        # ...
```

Putting everything together, we have the following circuit with two step types:

In [1]:
from chiquito.dsl import Circuit, StepType
from chiquito.cb import eq
from chiquito.util import F

class FiboFirstStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a, 1))
        self.constr(eq(self.circuit.b, 1))
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")
        
        self.fibo_first_step = self.step_type(FiboFirstStep(self, "fibo_first_step"))
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.pragma_num_steps(4)
        self.pragma_first_step(self.fibo_first_step)
        
    def trace(self, args):
        self.add(self.fibo_first_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 4):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

fibo = Fibonacci()
fibo_witness = fibo.gen_witness(None)
fibo.halo2_mock_prover(fibo_witness)

318146001539755052364393543636821346826
Ok(
    (),
)


We construct and pass over the same `evil_witness` from Chapter 3. This time `evil_witness` fails at the two new constraints we wrote, e.g. `a == 1` and `b == 1` from `FiboFirstStep`.

In [2]:
evil_witness = fibo_witness.evil_witness_test(step_instance_indices=[0, 0, 1, 1, 2, 2, 3, 3, 3], assignment_indices=[0, 1, 0, 2, 1, 2, 0, 1, 2], rhs=[F(0), F(2), F(2), F(4), F(4), F(6), F(4), F(6), F(10)])
fibo.halo2_mock_prover(evil_witness)

Err(
    [
        ConstraintCaseDebug {
            constraint: Constraint {
                gate: Gate {
                    index: 0,
                    name: "main",
                },
                index: 3,
                name: "fibo_first_step::(a == 1) => (a + -0x15ebf95182c5551cc8260de4aeb85d5d090ef5a9e111ec87dc5ba0056db1194e) => Product(Fixed { query_index: 0, column_index: 0, rotation: Rotation(0) }, Product(Advice { query_index: 6, column_index: 4, rotation: Rotation(0) }, Sum(Advice { query_index: 1, column_index: 0, rotation: Rotation(0) }, Negated(Constant(0x15ebf95182c5551cc8260de4aeb85d5d090ef5a9e111ec87dc5ba0056db1194e)))))",
            },
            location: InRegion {
                region: Region 0 ('circuit'),
                offset: 0,
            },
            cell_values: [
                (
                    DebugVirtualCell {
                        name: "",
                        column: DebugColumn {
                            column_type: Adv

By adding new constraints to eliminate soundness errors, this chapter demonstrated an iterative way of developing zero knowledge circuits. Circuit engineer can proactively generate witnesses in edge cases and if a false positive passes the circuit, add more constraints to prevent these edge cases. 

Besides, we learned that with multiple step types, Chiquito can create step instances with different signals and constraints, thereby allowing great versatility in circuit design.

Finally, this example shows again that circuit setup and witness generation are separate processes. There can be multiple witnesses for the same circuit setup. As a great analogy, a circuit can be considered a piece of "hardware" whereas its witnesses "softwares" that are compatible with the hardware.